# Libraries

In [55]:
# standard
import pandas as pd
import numpy as np
from tqdm import tqdm

# finance
import yfinance as yf
!git clone https://github.com/joqueka/bf4py.git
import sys
sys.path.append("bf4py")
from bf4py import BF4Py

# efficiency
from concurrent.futures import ThreadPoolExecutor

Cloning into 'bf4py'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 76 (delta 39), reused 60 (delta 27), pack-reused 0
Receiving objects: 100% (76/76), 25.28 KiB | 1.05 MiB/s, done.
Resolving deltas: 100% (39/39), done.


# Data Import

### get list of all German listed companies

https://topforeignstocks.com/stock-lists/the-list-of-listed-companies-in-germany/

In [26]:
traded_companies_path = '../data/Complete-List-of-Listed-Companies-on-Germany-Frankfurt-Stock-Exchange-Jan-2021.xlsx'

traded_companies = pd.DataFrame()

# List the sheet names you want to read
sheet_names_to_read = [1, 2, 3]  # 0-based index, or you can use sheet names as strings

# Loop through each sheet to read and combine data
for sheet_name in sheet_names_to_read:
    # Read the sheet into a DataFrame, skipping the first 7 rows
    df = pd.read_excel(traded_companies_path, sheet_name=sheet_name, skiprows=range(0, 7), usecols=range(0,6))
    
    # Concatenate the newly read DataFrame to the combined DataFrame
    traded_companies = pd.concat([traded_companies, df], ignore_index=True, axis=0)

# only keep german companies
traded_companies = traded_companies[traded_companies['Country'] == 'Germany']

traded_companies.head()


,ISIN,Trading Symbol,Company,Sector,Subsector,Country
0,DE0005545503,DRI,1+1 DRILLISCH AG O.N.,Telecommunication,Wireless Communication,Germany
1,DE0005118806,TGT,11 88 0 SOLUTIONS AG,Telecommunication,Telecommunication Services,Germany
2,DE0005167902,UUU,3U HOLDING AG,Industrial,Industrial Products & Services,Germany
3,DE000A2YN801,4BSB,4BASEBIO AG NA O.N.,Pharma & Healthcare,Biotechnology,Germany
4,DE000A14KL72,VSC,4SC AG,Pharma & Healthcare,Biotechnology,Germany


### get historical data

In [52]:
import yfinance as yf

# Convert the Series to a list
traded_companies_list = traded_companies['Trading Symbol'].astype(str).tolist()

# Download the data
stock_prices = yf.download(traded_companies_list, group_by='column', period='1mo')

# Drop all but the 'Close' columns
stock_prices = stock_prices.xs('Close', axis=1, level=0).dropna(axis=1)

[************          24%%                      ]  108 of 449 completed

Failed to get ticker 'TIMA' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)
Failed to get ticker 'BEZ' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)
Failed to get ticker 'DAM' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)


[************          24%%                      ]  109 of 449 completed

Failed to get ticker 'IFA' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)


[************          26%%                      ]  116 of 449 completed

...

Failed to get ticker 'SZG' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/SZG?range=1d&interval=1d (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff2d8af8310>: Failed to establish a new connection: [Errno 101] Network is unreachable'))


[*************         27%%                      ]  120 of 449 completed

Failed to get ticker 'WAF' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/WAF?range=1d&interval=1d (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff2d8097690>: Failed to establish a new connection: [Errno 101] Network is unreachable'))
Failed to get ticker 'VBX' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/VBX?range=1d&interval=1d (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff2d7f48b50>: Failed to establish a new connection: [Errno 101] Network is unreachable'))
Failed to get ticker 'D6H' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/D6H?range=1d&interval=1d (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff2d65e35d0>: Failed to establish a new connection: [Errno 10

KeyboardInterrupt: 

[*************         27%%                      ]  121 of 449 completed

In [ ]:
stock_prices.head()

,02P,1COV,2GB,2HRA,333,4BSB,4DS,5UH,8S9,8TRA,...,WUG,WUG3,WUW,WWG,YMO,YOC,YSN,ZAL,ZIL2,ZO1
Date,,,,,,,,,,,,,,,,,,,,,


[**************        30%%                      ]  133 of 449 completed

Failed to get ticker 'OBS' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)


[**************        30%%                      ]  134 of 449 completed

Failed to get ticker 'CSQ' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)
Failed to get ticker '4DS' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)


[*********************100%%**********************]  447 of 449 completed

Failed to get ticker 'PNE3' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)


[*********************100%%**********************]  448 of 449 completed

# Test Area

In [37]:
len(failed_downloads)

449

In [48]:
yf.download(['5UH', 'ML2'], period = '1mo')

[*********************100%%**********************]  2 of 2 completed


2 Failed downloads:
['ML2', '5UH']: Exception('%ticker%: No data found, symbol may be delisted')


Empty DataFrame
Columns: [(Adj Close, 5UH), (Adj Close, ML2), (Close, 5UH), (Close, ML2), (High, 5UH), (High, ML2), (Low, 5UH), (Low, ML2), (Open, 5UH), (Open, ML2), (Volume, 5UH), (Volume, ML2)]
Index: []